In [ ]:
import boto3
from botocore.exceptions import ClientError


def get_secret(region_name):
    secret_name = "teamphan-svc-account"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    # Decrypts secret using the associated KMS key.
    return get_secret_value_response['SecretString']


def get_region_name() -> str:
    return boto3.DEFAULT_SESSION.region_name \
        if boto3.DEFAULT_SESSION \
        else boto3.Session().region_name

In [ ]:
import json

secret = json.loads(get_secret(get_region_name()))

AWS_ACCESS_KEY = secret["AWS_ACCESS_KEY"]
AWS_SECRET_KEY = secret["AWS_SECRET_KEY"]
AWS_REGION = get_region_name()

athena_client = boto3.client(
    "athena",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)


In [ ]:
import time

S3_STAGING_DIR = "s3://teamphan-athena-results/"

query_response = athena_client.start_query_execution(
    QueryString="SELECT distinct Run, variation FROM annotated_variations WHERE variation = 'E484K' LIMIT 10",
    QueryExecutionContext={"Database": "ncbi-vcf-codeathon-rc-db1"},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

query_status={}
while True:
    queryExecution = athena_client.get_query_execution(QueryExecutionId=query_response['QueryExecutionId'])
    query_status = queryExecution["QueryExecution"]["Status"]
    if query_status["State"] in ['QUEUED', 'RUNNING']:
        time.sleep(0.001)
    else:
        break

if "State" in query_status:
    print(query_status["State"], "\n")
if "StateChangeReason" in query_status:
    print("StateChangeReason", query_status["StateChangeReason"], "\n")
if 'AthenaError' in query_status:
    print("An error occurred: ", query_status['AthenaError'])
